<a href="https://colab.research.google.com/github/MirasKaidullayev/NLP/blob/main/NLP_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a execution of chatbot that is intented to help companies hire people 

In [1]:
pip install tflearn 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.3/107.3 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=168c1ff7c0c68ed82e1e4ea52da31549a252a08d5ce0ffe97c665acb36ddb59f
  Stored in directory: /root/.cache/pip/wheels/55/fb/7b/e06204a0ceefa45443930b9a250cb5ebe31def0e4e8245a465
Successfully built tflearn


In [2]:
#Used in Tensorflow Model
import numpy as np
import tensorflow as tf
import tflearn
import random

#Usde to for Contextualisation and Other NLP Tasks.
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
print("Processing the Intents.....")
with open('/content/Intends.txt') as json_data:
    intents = json.load(json_data)

Processing the Intents.....


In [7]:
print(intents)

{'intents': [{'tag': 'interested', 'patterns': ['I am interested', 'interested', 'yes'], 'responses': ["Let's conduct a online job interview. We have 2 available slots. What is convinient for you? Today at 16:00 or tomorrow at 16:00."], 'context_set': ''}, {'tag': 'not interested', 'patterns': ['I am not interested', 'not interested', 'no', 'nope', 'I already have a job'], 'responses': ['Okay, have a good day']}, {'tag': 'today', 'patterns': ['Today is conviniet', 'today', 'I can today', "Let's have it today"], 'responses': ['Great we are waiting you today at zoom meeting 16:00 for a online job interview. The link for a meeting is: XXX. You can also ask question about the vacancy here']}, {'tag': 'tomorrow', 'patterns': ['Tomorrow is conviniet', 'tomorrow', 'I can tomorrow', "Let's have it tomorrow"], 'responses': ['Great, we are waiting you tomorrow at zoom meeting 16:00 for a online job interview. The link for a meeting is: XXX. You can also ask question about the vacancy here']}, {'

In [5]:
import nltk
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [9]:
words = []
classes = []
documents = []
ignore_words = ['?']
print("Looping through the Intents to Convert them to words, classes, documents and ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Looping through the Intents to Convert them to words, classes, documents and ignore_words.......


In [10]:
print("Preprosessing")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Preprosessing
46 documents
12 classes ['can not', 'company', 'goodbye', 'interested', 'location', 'not interested', 'requirements', 'responsibilities', 'salary', 'thanks', 'today', 'tomorrow']
52 unique stemmed words ["'s", 'a', 'about', 'adress', 'already', 'am', 'and', 'ar', 'busy', 'bye', 'can', 'company', 'conviniet', 'conviny', 'do', 'doe', 'for', 'goodby', 'hav', 'help', 'how', 'i', 'interest', 'is', 'it', 'job', 'lat', 'let', 'loc', 'meet', 'mor', 'much', 'no', 'nop', 'not', 'off', 'pay', 'requir', 'respons', 'sal', 'see', 'tel', 'thank', 'that', 'the', 'today', 'tomorrow', 'whar', 'what', 'wher', 'ye', 'you']


In [11]:
print("Creating the Data for our Model")
training = []
output = []
print("Creating an List (Empty) for Output")
output_empty = [0] * len(classes)

print("Creating Traning Set, Bag of Words for our Model")
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Creating the Data for our Model
Creating an List (Empty) for Output
Creating Traning Set, Bag of Words for our Model


In [12]:
print("Shuffling Randomly and Converting into Numpy Array for Faster Processing")
random.shuffle(training)
training = np.array(training)

print("Creating Train and Test Lists")
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Building Neural Network for Out Chatbot to be Contextual")
print("Resetting graph data")
tf.compat.v1.reset_default_graph()

Shuffling Randomly and Converting into Numpy Array for Faster Processing
Creating Train and Test Lists
Building Neural Network for Out Chatbot to be Contextual
Resetting graph data


In [13]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
print("Training....")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Training....


In [16]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [17]:
print("Training the Model.......")
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
print("Saving the Model.......")
model.save('model.tflearn')

Training Step: 5999  | time: 0.021s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 1.0000 -- iter: 40/46
Training Step: 6000  | time: 0.025s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 1.0000 -- iter: 46/46
--
Saving the Model.......


In [18]:
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [20]:
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('/content/Intends.txt') as json_data:
    intents = json.load(json_data)
    

# load our saved model
model.load('./model.tflearn')

In [21]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

ERROR_THRESHOLD = 0.25


In [22]:
print("Greetings, my name is Miras. I am hiring manager from company XXX. We are seeking a highly motivated data scientist to join our team. You must have submitted your resume for Data Science position? Are you still interested in this vacancy?")
while True:
    input_data = input("You- ")
    answer = response(input_data)
    answer

Greetings, my name is Miras. I am hiring manager from company XXX. We are seeking a highly motivated data scientist to join our team. You must have submitted your resume for Data Science position? Are you still interested in this vacancy?
You- I am interested
Let's conduct a online job interview. We have 2 available slots. What is convinient for you? Today at 16:00 or tomorrow at 16:00.
You- Today is conviniet
Great we are waiting you today at zoom meeting 16:00 for a online job interview. The link for a meeting is: XXX. You can also ask question about the vacancy here
You- What is the company
We are a banking company XXX. You can get more information about our company by following this link: XXX
You- what are the job requirements
Strong programming skills in languages such as Python, R, and SQL. Experience with data preprocessing, data cleaning, and feature engineering. Experience with machine learning algorithms such as regression, classification, and clustering.
You- how much do you

KeyboardInterrupt: ignored